In [38]:
import pandas as pd
import numpy as np
import pickle

# get the money flow between daos
Criteria: if a transaction is targeted to "x.sputnikdao.near" or "x.sputnik-dao.near", then it's a between-dao transaction.

In [91]:
fdir = 'data/'
fname = 'Sputnik DAO Proposals.xlsx'
df_dao_proposals = pd.read_excel(fdir+fname,sheet_name=None)
df_out={'dao_give':[]}
df_in={'dao_receive':[]}
df_alltrans={'dao_give':[],'dao_receive':[],'amount':[]}
for daoname in df_dao_proposals.keys():
    if daoname=='WORKSPACE': # this is not a dao, so skip it
        continue
    df = df_dao_proposals[daoname]
    df.columns = df.columns.str.replace(' ', '_').str.lower() # clean column names to avoid pandas key error
    
    # transform the amount to be in the unit of near
    if 'kind__amount' in df.keys():
        df['amount_(near)'] = pd.to_numeric(df['kind__amount'],errors = 'coerce')/(10**24)
       
    else:
        df['amount_(near)'] = pd.to_numeric(df['amount_(near)'],errors = 'coerce')
    
    # also, standardize column names
    if 'target' not in df.columns:
        df.columns = df.columns.str.replace('.*recipient.*', 'target',regex=True)

    # look into the fund flow
    allgives = {'receiver':[],'amount':[]}
    for index, prop in df.iterrows():
        if prop['status']=='Success': # only counting successful transactions
            if prop['target'][-17:] == '.sputnik-dao.near':
        
                df_alltrans['dao_receive'].append( prop['target'][:-17])
            elif  prop['target'][-16:] == '.sputnikdao.near':
                df_alltrans['dao_receive'].append( prop['target'][:-16])
            else: 
                continue
            df_alltrans['dao_give'].append(daoname)
            df_alltrans['amount'].append( prop['amount_(near)'])
                
    
df_alltrans = pd.DataFrame(df_alltrans)

In [103]:
df_alltrans.head()

,dao_give,dao_receive,amount
0,Community Squad,createbase,300.0
1,Community Squad,portuguese,150.0
2,Community Squad,sandbox,2551.0
3,Community Squad,nft-gaming,2000.0
4,Community Squad,vr-dao,2000.0


In [143]:
df_alltrans.to_csv('transactions_between_daos.csv')

# Question 1: how much of the funding comes from community squad?

In [45]:
daogive = 'Community Squad'
comsquad_give = df_alltrans.loc[df_alltrans['dao_give']==daogive].groupby('dao_receive').sum().sort_values(by='amount')
print(comsquad_give)

                               amount
dao_receive                          
neararabia                      43.50
createbase                     300.00
afrikaburn                     428.00
nearsamurai                    490.00
bogotahacknode                 560.00
san-francisco-near-hacknode    700.00
near_asia_dev_dao              710.00
lisboa-node                    913.00
nyc                           1000.00
near-data-center              1000.00
nft-onboarding                1005.00
lisboa-hack                   1151.95
npk-guild-nigeria             1367.50
portuguese                    1395.00
telegramcommunity             1600.00
mukachevo                     1885.00
near_india_guild              1943.50
nft-gaming                    2000.00
art                           2000.00
vr-dao                        2000.00
curators                      2100.00
misfits                       4142.00
wiki                          4500.00
rucommunity                   5250.00
vitalpointai

conclusion 1: not all. many daos like human guild, degen, NxM do not receive funding from community squad (so how to track their income?)

## for daos receiving money from community squad, is that their major source?

In [94]:
# load the current value of each dao
df_daostats = pickle.load(open('dao_stats','rb'))
df_daostats['Name']=df_daostats['Name'].str.replace('.sputnikdao.near','',regex=True)

df_daostats.head()

,Name,Ⓝ Value,Value (USD),Proposals,In Progress,Successful,Failed,Expired
0,genesis,15073.0,157965.0,45.0,0.0,15.0,9.0,21.0
1,academy,48.0,503.0,5.0,0.0,2.0,0.0,3.0
2,ethdenver,264.0,2767.0,133.0,0.0,92.0,12.0,28.0
3,arts,205.0,2144.0,8.0,0.0,7.0,0.0,1.0
4,createbase,503.0,5272.0,483.0,4.0,370.0,77.0,22.0


In [95]:
# load the total funding distributed historically
df_grants = pickle.load(open('dao_prop_stats','rb'))
df_grants.head()

,dao,grants,total_proposal,total_success_proposal,first_prop_time,last_prop_time,proposal_freq
12,Grants,1700.000000,5,1,1.616817e+18,1.621880e+18,0.085321
11,Bounties,4986.000000,8,6,1.630957e+18,1.634140e+18,0.217143
8,Genesis,5928.000000,45,15,1.610937e+18,1.625782e+18,0.261898
10,NxM,7386.751634,106,85,1.617318e+18,1.637067e+18,0.463734
2,Marketing Vertical,20724.000000,71,35,1.630059e+18,1.637152e+18,0.864883


In [96]:
bigdaos = ['Creatives','Createbase','Marketing Vertical','Open Web Sandbox']
for dao in bigdaos:
    daoalt = dao.lower().replace(' ','-')
    if dao == 'Open Web Sandbox':
        daoalt = 'sandbox'
    print("{} has total fund {}, {} comes from community squad.".format(dao,float(df_grants.loc[df_grants['dao']==dao]['grants'])+float(df_daostats.loc[df_daostats['Name']==daoalt]['Ⓝ Value']),comsquad_give.loc[daoalt]['amount']))
    

Creatives has total fund 82512.02, 62434.0 comes from community squad.
Createbase has total fund 67297.65377887999, 300.0 comes from community squad.
Marketing Vertical has total fund 21499.0, 20831.14 comes from community squad.
Open Web Sandbox has total fund 73291.17000000006, 73271.0 comes from community squad.


conclusion: some has most of its funding from community squad(marketing vertical, sandbox), some has the majority (Creatives), some only a tiny bit (Createbase)

# Question 2: if it's not from community squad, can we find all the sources of funds?

## create base

In [100]:
dao = 'createbase'
comsquad_give = df_alltrans.loc[df_alltrans['dao_receive']==dao].groupby('dao_give').sum().sort_values(by='amount')
print(comsquad_give)

                 amount
dao_give               
Createbase         20.0
Community Squad   300.0
Creatives        7710.0


## creatives

In [101]:
dao = 'creatives'
comsquad_give = df_alltrans.loc[df_alltrans['dao_receive']==dao].groupby('dao_give').sum().sort_values(by='amount')
print(comsquad_give)

# note: there's money from creatives to creatives for "Moving funds from Sputnik to Astro!"

                   amount
dao_give                 
Creatives        31190.58
Community Squad  62434.00


## degens

In [105]:
dao = 'degens'
comsquad_give = df_alltrans.loc[df_alltrans['dao_receive']==dao].groupby('dao_give').sum().sort_values(by='amount')
print(comsquad_give)

Empty DataFrame
Columns: [amount]
Index: []


## NxM

In [130]:
df_trans_receive = df_alltrans.groupby(['dao_receive','dao_give']).sum()

In [144]:
df_trans_receive.to_csv('transaction_receiver_sums.csv')

In [131]:
df_trans_receive.loc['nxm'] # can't find other source of income for nxm. this is for transfering to astro, again.

,amount
dao_give,
NxM,420.0


In [134]:
# wait actaully, some nxm are under "nxm-dao"
df_trans_receive.loc['nxm-dao'] # can't find other source of income for nxm. this is for transfering to astro, again.

,amount
dao_give,
Creatives,4205.0
Marketing Vertical,140.0


In [138]:
# how much of nxm dao is from creatives / marketing vertical compared to their total funds?
dao='NxM'
daoalt = dao.lower()+'-dao'
    
print("{} has total fund {}".format(dao,float(df_grants.loc[df_grants['dao']==dao]['grants'])+float(df_daostats.loc[df_daostats['Name']==daoalt]['Ⓝ Value'])))

# okay, still a large chunk of income missing

NxM has total fund 7889.751633986928


conclusion: still missing a bit chunk of money source.

## in general, who are the significant money givers?

In [139]:
df_give_sum = df_alltrans.groupby(['dao_give']).sum()

In [142]:
df_give_sum.sort_values(by='amount',ascending=False)

,amount
dao_give,
Community Squad,221620.59
Creatives,70137.02
Move Capital,48001.00
Ecosystem (KoG),28900.00
Human Guild,19500.00
Createbase,7768.00
Marketing Vertical,3950.00
NxM,1266.00
Genesis,333.00
